<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import scipy.stats
from scipy.stats import ttest_ind

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
                header=None,
                names=['party','handicapped-infants','water-project',
                      'budget', 'physician-fee-freeze','el-salvador',
                      'religion-groups','anti-satellite-ban',
                      'aid-to-contras','mx-missile','immigration',
                      'synfuels','education','right-to-sue','crime','duty-free',
                      'south-africa'], na_values='?')
df.head()


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador,religion-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [3]:
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador              15
religion-groups          11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [4]:
df = df.replace({'n':0,'y':1})
# Making the data numerical

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador,religion-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
rep = df[df.party == 'republican']
dem = df[df.party == 'democrat']
# Split by party

In [7]:
ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'], nan_policy='omit')
# ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'], nan_policy='omit')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [35]:
for issue in range(1,df.shape[1]):
  print(ttest_ind(rep.iloc[:,issue], dem.iloc[:,issue], nan_policy='omit'))
  print(df.columns.values[issue])
  print('\n')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
handicapped-infants


Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
water-project


Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
budget


Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
physician-fee-freeze


Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
el-salvador


Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
religion-groups


Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
anti-satellite-ban


Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)
aid-to-contras


Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)
mx-missile


Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)
immigration


Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)

1. With a p-value of about two times ten to the seventy-seventh power, democrats supported the budget more than republicans.
2. With a p-value of about two times ten to the 177th powers, republicans supported the freeze in physician feels more than democrats.
3. With a p-value above 0.9, there was no significant correlation between support for the water project and political party.

In [0]:
# positive = republican

In [0]:
# df1 = df.fillna(df.mode().iloc[0])
# df1.head()
# Fill NaN values with the most common value, presumably the winner of the vote